In [1]:
#@title Packages and functions
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import jv, jvp, yv, iv, spherical_jn
from scipy.optimize import root_scalar, brentq
plt.rcParams.update({'font.size': 22})
# %matplotlib qt

def Sphere_modes(cL, cT, R, order, modes):
  def Sato_Saviot_Lamb(omega, p, cl, ct, R):
      '''
      Basic Study on the Oscillation of Homogeneous Elastic Sphere-Part I: Frequency of the Free Oscillations
      Equation 2.2. 1962.
      
      The equations for the zero-order (n = 0) were modified according to Lamb (1881) based on corrections pointed out by 
      Saviot et al.(2004) in:
      - Saviot, L., Murray, D. B., Mermet, A., & Duval, E. (2004). Comment on “Estimate of the vibrational 
      frequencies of spherical virus particles”. Physical Review E, 69(2), 023901. [doi: https://doi.org/10.1103/PhysRevE.69.023901]
      - Lamb, H. (1881). On the vibrations of an elastic sphere. 
      Proceedings of the London Mathematical Society, 1(1), 189-212. [doi: https://doi.org/10.1112/plms/s1-13.1.189]
      omega = frequency
      p = angular wavenumber
      cl = longitudinal wave velocity of material
      ct = shear wave velocity of material
      '''
      # re-scaling for numerical convenience
      R = R*1000
      cl = cl/1000
      ct = ct/1000
      omega = omega/1e6
      # Code starts
      kl = omega/cl
      kt = omega/ct
      x = kl*R
      y = kt*R
      n = p #- (1/2)
      # Matrix form
      a = ((1/2) - n*(n-1)/y**2)*jv(n+(1/2),x) - (2*x/y**2)*jv(n+(3/2),x)
      b = -2*n*(n+1)*(((n - 1)/y**2)*jv(n+(1/2),x) - (x/y**2)*jv(n+(3/2),x))
      c = (((n-1)/y**2)*jv(n+(1/2),y) - (1/y)*jv(n+(3/2),y))
      d = (2/y)*jv(n+(3/2),y) + ((2*(n**2 -1)/y**2) - 1)*jv(n+(1/2),y)
      if n == 0:
          M = np.array([[np.tan(x),  4*x], [1, 4 - (x*cl/ct)**2]])  # Lamb (1882) Equation (59)        
      else:
          M = np.array([[a, b], [c, d]])
      return M

  def D_det(omega):
      '''Calculate the determinant of matrix '''
      return np.linalg.det(Sato_Saviot_Lamb(omega, kp, cL, cT, R))
  # Define constants and range of calculations
  fstop = 5.0*1e6 # Ending frequency in MHz # order*cL/R #
  dp = 1.0 
  wstart = 0.0001*1e6 # fstop/10000
  dw = 0.001*1e6 # fstop/1000 #
  kpstart = 0 
  kpstop = order # Order n

  omegas = np.arange(wstart,2*np.pi*fstop,dw)
  kps = np.arange(kpstart,kpstop,dp)

  number_of_roots = modes

  roots = []
  KP = []
  for kp in kps: 
      print('Order n: ', kp)
      print('mode', 'n', 'freq (Hz)', 'ka')
      c = 0 # counter for the uneven modes of order n=0
      rc = 0
      for omega in omegas:
          if kp < 1.0: 
              NR = number_of_roots 
              if rc < NR: #number_of_roots:
                  if (np.sign(D_det(omega)) > np.sign(D_det(omega+dw))) or (np.sign(D_det(omega)) < np.sign(D_det(omega+dw))):
                      root_val =  brentq(D_det,omega, omega+dw)
                      #root_val = root_scalar(D_det, bracket = [omega,omega+dw], method = 'bisect', xtol = 1e-7)# root_val.root
                      if root_val != None:
                          #roots.append(root_val.root)                          
                          roots.append(root_val)
                          KP.append(kp)                          
                          if rc % 2 != 0: # Print the even numbers of the zero-order
                              c = c + 1
                              print('%s, %d , %6.2f, %6.4f' % (''+str(int(kp))+'_S_'+str(c-1)+'', kp, root_val/(2*np.pi), root_val*R/cT))
                          #print('%s, %d , %6.2f, %6.4f' % (''+str(int(kp))+'_S_'+str(rc)+'', kp, root_val.root/(2*np.pi), root_val.root*R/cT))
                          rc = rc + 1
              elif rc == NR:
                  break
          elif kp >= 1.0: 
              NR = number_of_roots #
              if rc < NR: #number_of_roots:
                  if (np.sign(D_det(omega)) > np.sign(D_det(omega+dw))) or (np.sign(D_det(omega)) < np.sign(D_det(omega+dw))):
                      #print(omega)\
                      root_val =  brentq(D_det,omega, omega+dw)
                      #root_val = root_scalar(D_det, bracket = [omega,omega+dw], method = 'bisect', xtol = 1e-7)# root_val.root
                      if root_val != None:
                          #roots.append(root_val.root)
                          roots.append(root_val)
                          KP.append(kp)
                          print('%s, %d , %6.2f, %6.4f' % (''+str(int(kp))+'_S_'+str(rc)+'', kp, root_val/(2*np.pi), root_val*R/cT))
                          #print('%s, %d , %6.2f, %6.4f' % (''+str(int(kp))+'_S_'+str(rc)+'', kp, root_val.root/(2*np.pi), root_val.root*R/cT))
                          rc = rc + 1                         
              elif rc == NR:
                  break
  return roots, KP

# Calculate the fundamental eigenmodes of a sphere

You can compare the values to the ones of Lucien Saviot [here](https://saviot.cnrs.fr/lamb/index.en.html). Just be mindful that the overtones in this notebook start at index 0 instead of 1.

In this notebook, frequency intervals for root searching are tailored for ultrasonic frecuencies on spherical objects of radii on the centimeter scale, you can modify the code for more general purposes. 

The fundamental modes for order $n = 0$ are a special case covered by Lamb (1881).   Saviot et al. (2004) is a comment to remind us that in a homogeneous sphere:

> For materials with positive Poisson ratio, it is impossible to have the energy for the fundamental $n = 0$ mode smaller than the energy for the fundamental $n = 2$ one.

## References:

- Lamb, H. (1881). On the vibrations of an elastic sphere. Proceedings of the London Mathematical Society, 1(1), 189-212. [doi: https://doi.org/10.1112/plms/s1-13.1.189](https://doi.org/10.1112/plms/s1-13.1.189) 

- Sato, Y. and Usami, T. (1962). Basic study on the oscillation of a homogeneous elastic sphere-Part I: Frequency of the Free Oscillations, Geophysical Magazine, 31(15-24).

- Saviot, L., Murray, D. B., Mermet, A., & Duval, E. (2004). Comment on “Estimate of the vibrational frequencies of spherical virus particles”. Physical Review E, 69(2), 023901. [doi: https://doi.org/10.1103/PhysRevE.69.023901](https://doi.org/10.1103/PhysRevE.69.023901)

In [2]:
#@title Calculate fundamental modes

import ipywidgets as widgets
from IPython.display import display, clear_output

# Create input widgets
cL_widget = widgets.FloatText(value=6009.0, description='cL (m/s):')
cT_widget = widgets.FloatText(value=3212.0, description='cT (m/s):')
R_widget = widgets.FloatText(value=0.010, description='R (m):')
order_widget = widgets.FloatText(value=3, description='order:')
modes_widget = widgets.FloatText(value=10, description='modes:')

# Create button widget
button = widgets.Button(description="Calculate Modes")

# Create output widget
out = widgets.Output()

# Define on_button_clicked function
def on_button_clicked(b):
  with out:
    clear_output()
    roots, KP = Sphere_modes(cL_widget.value, cT_widget.value, R_widget.value, order_widget.value, modes_widget.value)
  return roots, KP
# Attach on_click event to button
button.on_click(on_button_clicked)

# Display widgets
display(order_widget, modes_widget, cL_widget, cT_widget, R_widget, button, out)

roots, KP = on_button_clicked(button)

FloatText(value=3.0, description='order:')

FloatText(value=10.0, description='modes:')

FloatText(value=6009.0, description='cL (m/s):')

FloatText(value=3212.0, description='cT (m/s):')

FloatText(value=0.01, description='R (m):')

Button(description='Calculate Modes', style=ButtonStyle())

Output()